In [1]:
# Nice Formatting within Jupyter Notebook
%matplotlib inline
#from IPython.display import display # Allows multiple displays from a single code-cell

# System functionality
import sys
sys.path.append('..')

# Import Here any Additional modules you use. To import utilities we provide, use something like:
#   from utils.plotter import plot_hinton

# Your Code goes here:
#from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split #, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#from sklearn import preprocessing
from numpy import unravel_index
from matplotlib.colors import ListedColormap
from numpy import linspace
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
#####################
###   load data   ###
#####################

# load datasets
data_path = "/home/philipp/Documents/boku/remote_sensing_and_image_processing/final_ass/train_data_SX.csv"
train = pd.read_csv(data_path, delimiter = ',', index_col=0)
data_path = "/home/philipp/Documents/boku/remote_sensing_and_image_processing/final_ass/test_data.csv"
test = pd.read_csv(data_path, delimiter = ',', index_col=0)

train = shuffle(train, random_state=399)

X_train = train.drop(['crop_id','geometry'], axis=1).values
y_train = train['crop_id'].values
X_test = test.drop(['crop_id','geometry'], axis=1).values
y_test = test['crop_id'].values

print("Train: number of instances: {}, number of attributes: {}".format(train.shape[0], train.shape[1]))
print("Test: number of instances: {}, number of attributes: {}".format(test.shape[0], test.shape[1]))

Train: number of instances: 1200, number of attributes: 215
Test: number of instances: 4584, number of attributes: 101


In [3]:
##########################
###   exploring data   ###
##########################

print(train.head(7))

train.describe()

      crop_id                                     geometry  B02_0322  \
id                                                                     
168         2  POINT (634740.5100236891 5339978.909767563)       640   
669        10   POINT (624908.0456842275 5335769.90775792)       461   
1157       15  POINT (621332.3669208327 5360109.999892802)       609   
264         6  POINT (639971.0221898934 5343010.385524059)       532   
194         2   POINT (639366.526787769 5338334.476987346)       697   
679        10  POINT (620109.7202541821 5354703.312341657)       466   
327         3  POINT (632185.2000834884 5348559.848026753)       645   

      B03_0322  B04_0322  B05_0322  B06_0322  B07_0322  B08A_0322  B08_0322  \
id                                                                            
168        713       923      1054      1199      1347       1503      1457   
669        516       651       686       760       833        933       891   
1157       742       978      1179 

,crop_id,B02_0322,B03_0322,B04_0322,B05_0322,B06_0322,B07_0322,B08A_0322,B08_0322,B11_0322,...,VH_20180824,VV_20180824,VH_20180901,VV_20180901,VH_20180905,VV_20180905,VH_20180913,VV_20180913,VH_20180917,VV_20180917
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,...,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,8.500000,581.880833,708.495833,869.456667,1107.196667,1406.018333,1541.455833,1708.319167,1626.284167,2085.293333,...,-21.057349,-12.514461,-15.676784,-8.235546,-17.978829,-9.398661,-23.232206,-12.331709,-22.225848,-12.670772
std,4.892506,112.002018,145.255491,193.444519,232.888240,493.603601,548.194337,570.986702,570.295898,393.887780,...,4.801779,3.227872,3.239966,2.454739,4.053307,2.694878,7.064837,2.979832,4.947382,2.882676
min,1.000000,314.000000,323.000000,401.000000,491.000000,555.000000,664.000000,744.000000,670.000000,1024.000000,...,-47.360827,-26.088645,-28.093690,-18.262707,-47.377829,-18.701451,-49.906834,-23.226203,-47.336408,-22.106770
25%,4.500000,512.000000,616.750000,746.750000,943.750000,1073.750000,1170.750000,1323.000000,1252.500000,1828.500000,...,-24.000311,-14.523640,-17.495173,-9.802562,-20.173758,-11.066298,-26.230068,-14.240687,-24.970871,-14.627623
50%,9.000000,568.000000,692.000000,844.000000,1096.000000,1273.000000,1395.500000,1564.500000,1472.500000,2121.000000,...,-20.769847,-12.466210,-15.292160,-8.152490,-17.457946,-9.327234,-22.207835,-12.367720,-22.004615,-12.807948
75%,12.250000,634.250000,780.250000,967.250000,1242.000000,1611.250000,1755.250000,1924.250000,1838.750000,2352.500000,...,-17.515438,-10.396693,-13.384850,-6.661032,-15.237383,-7.643698,-18.282779,-10.308123,-18.769907,-10.811282
max,16.000000,1219.000000,1407.000000,1831.000000,2074.000000,3734.000000,4204.000000,4436.000000,4372.000000,3885.000000,...,-11.186590,-2.475107,-8.402266,3.603084,-10.440328,1.809863,-11.211261,-1.111857,-10.877446,-0.035265


In [4]:
############################
###   standardize data   ###
############################

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

In [ ]:
##########################
##########################
###                    ###
###   Classification   ###
###                    ###
##########################
##########################

In [ ]:
##########################################
###   RFC - Random Forest Classifier   ###
##########################################

In [9]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=399, n_jobs=8)
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(scores), scores)

0.7816666666666667 [0.775      0.80833333 0.81666667 0.775      0.775      0.75833333
 0.81666667 0.75       0.79166667 0.75      ]


In [6]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=399, n_jobs=8)
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(scores), scores)

0.9662308509909756 [0.97033013 0.96698705 0.96155453 0.9653155  0.96030088 0.96573339
 0.96782282 0.96906355 0.96948161 0.96571906]


In [10]:
param_grid = [
    {'n_estimators': [1, 10, 100, 1000], 'criterion': ['gini', 'entropy'], \
     'max_depth': [None, 10, 100, 1000], 'max_features': ['sqrt', 'log2', None], },
 ]

In [11]:
clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=8,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [None, 10, 100, 1000],
                          'max_features': ['sqrt', 'log2', None],
                          'n_estimators': [1, 10, 100, 1000]}])

In [12]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print("------------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 1000}

Best mean cross-validated score: 0.8391666666666666

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.600 (+/-0.076) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 1}
0.773 (+/-0.043) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 10}
0.824 (+/-0.053) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100}
0.837 (+/-0.061) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 1000}
0.592 (+/-0.095) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 1}
0.774 (+/-0.051) for {'criterion': 'gini', 'max_dept

In [6]:
param_grid = [
    {'n_estimators': [600,1000,3000], 'criterion': ['gini'], \
     'max_depth': [1000], 'max_features': ['sqrt'], },
 ]

In [7]:
clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [21]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print("------------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 600}

Best mean cross-validated score: 0.8358333333333332

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.836 (+/-0.045) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 600}
0.833 (+/-0.061) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 1000}
0.835 (+/-0.062) for {'criterion': 'gini', 'max_depth': 1000, 'max_features': 'sqrt', 'n_estimators': 3000}
------------------------------------------------------------------------------------------------------


In [ ]:
###########################################
###   SVC - Support Vector Classifier   ###
###########################################

In [5]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.8541666666666667 [0.825      0.86666667 0.85833333 0.875      0.83333333 0.85833333
 0.85833333 0.86666667 0.86666667 0.83333333]


In [8]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.8799999999999999 [0.88333333 0.9        0.86666667 0.90833333 0.88333333 0.90833333
 0.85833333 0.85       0.88333333 0.85833333]


In [7]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.9811511452718141 [0.98328458 0.98203092 0.98203092 0.97743418 0.97785207 0.98161304
 0.98035938 0.98620401 0.97993311 0.98076923]


In [5]:
clf = SVC(kernel='linear', C=1, random_state=399)
scores = cross_val_score(clf, X_train, y_train, cv=10, n_jobs=8)
print(np.mean(scores), scores)

0.8883333333333334 [0.89166667 0.89166667 0.89166667 0.90833333 0.9        0.89166667
 0.875      0.89166667 0.9        0.84166667]


In [6]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'degree': [2, 3, 4], 'kernel': ['poly']},
    {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'], 'kernel': ['sigmoid']},
 ]

In [7]:
clf = GridSearchCV(SVC(), param_grid, cv=10, n_jobs=8)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=8,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'degree': [2, 3, 4],
                          'gamma': ['scale', 'auto'], 'kernel': ['poly']},
                         {'C': [1, 10, 100, 1000], 'gamma': ['scale', 'auto'],
                          'kernel': ['sigmoid']}])

In [8]:
print("------------------------------------------------------------------------------------------------------")
print()
print("Best parameters set found on development set: ", clf.best_params_)
print()
print('Best mean cross-validated score:', clf.best_score_)
print()
print("------------------------------------------------------------------------------------------------------")
print("Grid scores for SVM:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

------------------------------------------------------------------------------------------------------

Best parameters set found on development set:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

Best mean cross-validated score: 0.9883333333333335

------------------------------------------------------------------------------------------------------
Grid scores for SVM:

0.971 (+/-0.009) for {'C': 1, 'kernel': 'linear'}
0.972 (+/-0.010) for {'C': 10, 'kernel': 'linear'}
0.968 (+/-0.010) for {'C': 100, 'kernel': 'linear'}
0.968 (+/-0.009) for {'C': 1000, 'kernel': 'linear'}
0.971 (+/-0.007) for {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.971 (+/-0.007) for {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
0.988 (+/-0.005) for {'C':

In [12]:
######################
###   Prediction   ###
######################

In [13]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [14]:
pred = clf.predict(X_test)

In [17]:
confusion_matrix(y_test, pred)

array([[355,   1,   1,  15,   2,  13,   3,   5,   0,   2,   2,   1],
       [  4, 372,   0,  18,   2,   0,   1,   0,   0,   3,   0,   0],
       [  0,   0, 380,   2,   0,   0,   0,   0,   8,   0,   9,   1],
       [ 12,   5,   5, 317,  14,   2,   3,   3,   0,   9,  23,   7],
       [  5,   7,   0,  18, 314,   4,  10,  21,   1,   7,  13,   0],
       [ 11,   3,   0,  11,   4, 362,   3,   2,   0,   3,   1,   0],
       [  1,   0,   0,   6,   3,   6, 374,   2,   0,   2,   1,   5],
       [  3,   0,   0,   7,  35,   1,   5, 346,   0,   3,   0,   0],
       [  0,   0,   9,   3,   0,   0,   0,   0, 374,   3,   9,   2],
       [  0,   2,   0,   3,   2,   1,   0,   3,   0, 372,   3,  14],
       [  3,   0,   6,   3,   3,   0,   0,   0,   1,   6, 358,   0],
       [  0,   0,   0,   1,   0,   0,   0,   0,   0,   4,   0, 199]])

In [18]:
accuracy_score(y_test, pred)

0.8994328097731239

In [9]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [10]:
pred = clf.predict(X_test)

In [11]:
confusion_matrix(y_test, pred)

array([[375,   0,   2,   6,   1,   8,   2,   3,   0,   1,   1,   1],
       [  3, 384,   0,   8,   2,   0,   1,   0,   0,   2,   0,   0],
       [  0,   0, 379,   3,   1,   1,   0,   0,  13,   0,   3,   0],
       [ 12,  12,   5, 320,  14,   2,   3,   6,   0,   9,  14,   3],
       [  3,  10,   0,  11, 334,   2,  10,  12,   0,   4,  14,   0],
       [ 12,   1,   0,   8,   4, 369,   2,   1,   0,   2,   1,   0],
       [  0,   0,   0,   4,   5,   2, 384,   2,   0,   3,   0,   0],
       [  1,   1,   0,   1,  20,   0,   1, 374,   0,   0,   2,   0],
       [  0,   0,   4,   4,   0,   0,   0,   0, 380,   6,   5,   1],
       [  1,   1,   0,   1,   0,   0,   0,   3,   0, 383,   3,   8],
       [  3,   0,   1,   2,   4,   0,   0,   0,   0,   1, 369,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 204]])

In [12]:
accuracy_score(y_test, pred)

0.9282286212914486

In [8]:
# prediction with the best preforming model
clf = SVC(kernel='rbf', C=10, gamma='scale', random_state=399)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [9]:
confusion_matrix(y_test, pred)

array([[375,   0,   2,   4,   1,  11,   1,   3,   0,   1,   1,   1],
       [  4, 384,   0,   6,   2,   0,   1,   1,   0,   2,   0,   0],
       [  0,   0, 385,   2,   1,   2,   0,   0,   6,   1,   3,   0],
       [ 17,  12,   6, 319,  15,   2,   1,   6,   1,   4,  15,   2],
       [  2,   9,   1,   9, 338,   1,   6,  12,   0,   5,  17,   0],
       [ 15,   1,   0,   9,   3, 366,   2,   1,   1,   1,   1,   0],
       [  0,   0,   0,   3,   3,   2, 386,   2,   0,   4,   0,   0],
       [  1,   0,   0,   3,  19,   0,   4, 366,   0,   1,   6,   0],
       [  0,   0,   4,   3,   0,   0,   0,   0, 379,   6,   6,   2],
       [  2,   0,   0,   2,   0,   1,   0,   2,   0, 381,   4,   8],
       [  3,   0,   1,   0,   5,   0,   0,   0,   0,   0, 371,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0, 203]])

In [10]:
accuracy_score(y_test, pred)

0.9277923211169284

In [11]:
X_train.shape

(23927, 99)